In [1]:
import pandas as pd
import numpy as np
from LinearModel_exercise_pipes import *
from mypipes import *
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso,Ridge
from sklearn.model_selection import GridSearchCV

In [2]:
train=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/3rd PROJECT/counterfeit_train.csv')

In [3]:
test=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/3rd PROJECT/counterfeit_test.csv')

In [ ]:
train.head(5)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Medicine_ID          6818 non-null   object 
 1   Counterfeit_Weight   5652 non-null   float64
 2   DistArea_ID          6818 non-null   object 
 3   Active_Since         6818 non-null   int64  
 4   Medicine_MRP         6818 non-null   float64
 5   Medicine_Type        6818 non-null   object 
 6   SidEffect_Level      6818 non-null   object 
 7   Availability_rating  6818 non-null   float64
 8   Area_Type            6818 non-null   object 
 9   Area_City_Type       6818 non-null   object 
 10  Area_dist_level      6818 non-null   object 
 11  Counterfeit_Sales    6818 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 639.3+ KB


In [5]:
for col in ['Medicine_ID','Counterfeit_Weight']:
    train.drop(col,1,inplace=True)
    test.drop(col,1,inplace=True)

In [6]:
for col in ['Medicine_Type','SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level',"DistArea_ID"]:  
    temp=pd.get_dummies(train[col],prefix=col,drop_first=True)
    train=pd.concat([temp,train],1)
    train.drop([col],1,inplace=True)
       
    temp=pd.get_dummies(test[col],prefix=col,drop_first=True)
    test=pd.concat([temp,test],1)
    test.drop([col],1,inplace=True)

In [7]:
target='Counterfeit_Sales'

In [8]:
x_train=train.drop(target,1)
y_train=train[target]

# Model Fitting

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [10]:
param_dist = {"n_estimators":[50,100,200,300,400],
              "max_features": [2,4,5,6,8],
              "bootstrap": [True, False],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

In [11]:
reg=RandomForestRegressor()

In [12]:
random_search=RandomizedSearchCV(reg,
                                 cv=10,
                                 param_distributions=param_dist,
                                 scoring='neg_mean_absolute_error',
                                 n_iter=10,n_jobs=-1,verbose=20
                                    )

In [13]:
random_search.fit(x_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      50, 70],
                                        'max_features': [2, 4, 5, 6, 8],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [50, 100, 200, 300,
                                                         400]},
                   scoring='neg_mean_absolute_error', verbose=20)

In [14]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [15]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: -765.11620 (std: 24.98967)
Parameters: {'n_estimators': 400, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 8, 'max_depth': None, 'bootstrap': False}

Model with rank: 2
Mean validation score: -765.88445 (std: 24.79782)
Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 8, 'max_depth': 50, 'bootstrap': False}

Model with rank: 3
Mean validation score: -771.17886 (std: 26.76277)
Parameters: {'n_estimators': 50, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_features': 8, 'max_depth': 30, 'bootstrap': False}

Model with rank: 4
Mean validation score: -775.62258 (std: 26.37613)
Parameters: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 6, 'max_depth': 30, 'bootstrap': True}

Model with rank: 5
Mean validation score: -783.97993 (std: 25.31140)
Parameters: {'n_estimators': 300, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 5,

In [16]:
forest=RandomForestRegressor(**{'n_estimators': 100, 'min_samples_split': 15, 'min_samples_leaf': 15, 'max_features': 8, 'max_depth': 50, 'bootstrap': False})

In [17]:
forest.fit(x_train,y_train)

RandomForestRegressor(bootstrap=False, max_depth=50, max_features=8,
                      min_samples_leaf=15, min_samples_split=15)

In [18]:
submissions=pd.DataFrame({target:forest.predict(test)})

In [19]:
submissions.to_csv('sample_submission.csv',index=False)